In [4]:
# Importing all necessary libraries
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import os
import regex as re
from src.Logger import logger  # Import your logger instance from src/Logger.py
from tabulate import tabulate

# 1. Download, Extract, and Log Transactions data from GitHub 

In [2]:
zip_url = "https://github.com/CapitalOneRecruiting/DS/raw/master/transactions.zip"
extract_folder = "data/"

if not os.path.exists(extract_folder): 
    os.makedirs(extract_folder)

response = requests.get(zip_url) 

if response.status_code == 200:
    try:
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            zip_ref.extractall(extract_folder)
        log_message = f"Saved the extracted data files from transaction.zip on github to {extract_folder}"
        logger.info(log_message)  # Log the information using your logger
    except Exception as e:
        log_message = f"Failed to extract contents: {str(e)}"
        logger.error(log_message)  # Log the error using your logger
else:
    log_message = f"Failed to download the ZIP file. Status code: {response.status_code}"
    logger.error(log_message)

# 2. Load Dataset in chunks

In [5]:
chunk_size = 10000
chunks = []

for chunk in pd.read_json('data/transactions.txt', lines=True, chunksize=chunk_size):
    chunks.append(chunk)

df_raw = pd.concat(chunks, ignore_index=True) # Concatenate the chunks into a single DataFrame


In [ ]:
# Make a copy of the original dataset for cleaning
df = df_raw.copy()

In [6]:
df.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,,0.0,,,,False,,,False,False
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,,0.0,,,,True,,,False,False
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,,0.0,,,,False,,,False,False
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,,0.0,,,,False,,,False,False
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,,0.0,,,,True,,,False,False


In [ ]:
# As using head(), I am not able to see data for all the columns as it is truncated in between I am using below function
def Display_all_columns_nrow(df, n): #Example 8 = Top 4 & Bottom 4 Rows with all column view on scroll
    with pd.option_context("display.max_rows", n, "display.max_columns", df.shape[1]): 
        display(df)
Display_all_columns_nrow(df,8)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.00,2016-08-13T14:27:32,98.55,Uber,US,US,02,01,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,,0.00,,,,False,,,False,False
1,737265056,737265056,5000,5000.00,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,01,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,,0.00,,,,True,,,False,False
2,737265056,737265056,5000,5000.00,2016-11-08T09:18:39,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,,0.00,,,,False,,,False,False
3,737265056,737265056,5000,5000.00,2016-12-10T02:14:50,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,,0.00,,,,False,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786359,732852505,732852505,50000,48785.04,2016-12-25T16:20:34,18.89,hulu.com,US,US,09,01,online_subscriptions,08/2023,2012-08-23,2012-08-23,939,939,3388,PURCHASE,,1214.96,,,,False,,,False,False
786360,732852505,732852505,50000,48766.15,2016-12-27T15:46:24,49.43,Lyft,US,US,02,01,rideshare,08/2025,2012-08-23,2012-08-23,936,936,3783,PURCHASE,,1233.85,,,,False,,,False,False
786361,732852505,732852505,50000,48716.72,2016-12-29T00:30:55,49.89,walmart.com,US,US,09,99,online_retail,07/2022,2012-08-23,2012-08-23,939,939,3388,PURCHASE,,1283.28,,,,False,,,False,False
786362,732852505,732852505,50000,48666.83,2016-12-30T20:10:29,72.18,Uber,US,US,05,01,rideshare,05/2024,2012-08-23,2012-08-23,939,939,3388,PURCHASE,,1333.17,,,,False,,,False,False


# 3. Data Structure information and basic changes

In [9]:
print("Raw data Dimensions (rows x columns):", f"{df.shape[0]} x {df.shape[1]}")
print("Raw data number of Records:",df.shape[0])
print("Raw data number of Field:",df.shape[1])

Raw data Dimensions (rows x columns): 786363 x 29
Raw data number of Records: 786363
Raw data number of Field: 29


In [10]:
df.echoBuffer.unique()

array([''], dtype=object)

Replace blank cell and any only space column to np.nan in complete dataset

In [11]:
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

Concise information about the DataFrame's structure and data types

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786363 entries, 0 to 786362
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   accountNumber             786363 non-null  int64  
 1   customerId                786363 non-null  int64  
 2   creditLimit               786363 non-null  int64  
 3   availableMoney            786363 non-null  float64
 4   transactionDateTime       786363 non-null  object 
 5   transactionAmount         786363 non-null  float64
 6   merchantName              786363 non-null  object 
 7   acqCountry                781801 non-null  object 
 8   merchantCountryCode       785639 non-null  object 
 9   posEntryMode              782309 non-null  object 
 10  posConditionCode          785954 non-null  object 
 11  merchantCategoryCode      786363 non-null  object 
 12  currentExpDate            786363 non-null  object 
 13  accountOpenDate           786363 non-null  o

Insights: 
1. The dataset contains a total of 29 features, with 6 of them being null we need to drop completely null columns.
2. Over here as we can see that the datatype for the column transactionDateTime, currentExpDate, accountOpenDate, and dateOfLastAddressChange are wrong we need to convert it from String to date_time.
3. Rest of features are numerical or categorical. (described in detailed later)

Drop 6 completely null columns to reduce dimensionality

In [13]:
all_null_columns = df.columns[df.isna().all()].tolist()
print(all_null_columns)

['echoBuffer', 'merchantCity', 'merchantState', 'merchantZip', 'posOnPremises', 'recurringAuthInd']


In [14]:
df.drop(all_null_columns, axis=1,inplace=True)

In [15]:
Display_all_columns_nrow(df,6)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.00,2016-08-13T14:27:32,98.55,Uber,US,US,02,01,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,0.00,False,False,False
1,737265056,737265056,5000,5000.00,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,01,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,True,False,False
2,737265056,737265056,5000,5000.00,2016-11-08T09:18:39,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786360,732852505,732852505,50000,48766.15,2016-12-27T15:46:24,49.43,Lyft,US,US,02,01,rideshare,08/2025,2012-08-23,2012-08-23,936,936,3783,PURCHASE,1233.85,False,False,False
786361,732852505,732852505,50000,48716.72,2016-12-29T00:30:55,49.89,walmart.com,US,US,09,99,online_retail,07/2022,2012-08-23,2012-08-23,939,939,3388,PURCHASE,1283.28,False,False,False
786362,732852505,732852505,50000,48666.83,2016-12-30T20:10:29,72.18,Uber,US,US,05,01,rideshare,05/2024,2012-08-23,2012-08-23,939,939,3388,PURCHASE,1333.17,False,False,False


Data Type Change

 
- transactionDateTime, currentExpDate, accountOpenDate, and dateOfLastAddressChange from object to date_time

In [16]:
columns_to_convert = ['transactionDateTime', 'currentExpDate', 'accountOpenDate', 'dateOfLastAddressChange']
for col in columns_to_convert:
    if col == 'currentExpDate':
        df[col] = pd.to_datetime(df[col], format='%m/%Y')
    else:
        df[col] = pd.to_datetime(df[col])

In [17]:
print(df[columns_to_convert].dtypes)

transactionDateTime        datetime64[ns]
currentExpDate             datetime64[ns]
accountOpenDate            datetime64[ns]
dateOfLastAddressChange    datetime64[ns]
dtype: object


In [18]:
print(set(df.dtypes))

{dtype('bool'), dtype('O'), dtype('int64'), dtype('float64'), dtype('<M8[ns]')}


 # 4. Data Brief Function

## The `Data_Brief` function provides a summarized overview of a dataset 
(Below results are for the modified df)

- This function generates a summary report for a given DataFrame (Modified dataset) containing information about its dimensions, column names, categorical column total unique values, numerical features, and value counts of categorical columns with a specified threshold, and then prints the summary in a tabulated format. The code is optimize to work for more the current number of columns.

In [19]:
def Data_Brief(df, th_value_count=10, nrow=10):
    #considering bool as well in categorical features
    categorical_columns = df.select_dtypes(include=['object', 'category','bool']).columns.tolist()
    numerical_columns = df.select_dtypes(include=['int', 'float']).columns.tolist()
    datetime_columns = df.select_dtypes(include=['datetime']).columns.tolist()
    n_numerical, n_categorical, n_datetime = len(numerical_columns), len(categorical_columns), len(datetime_columns)

    summary_data = []  # List to store the summary data
    summary_data.append(["Dimensions (rows x columns)", f"{df.shape[0]} x {df.shape[1]}"])

    all_columns = [', '.join(df.columns[i:i+nrow]) for i in range(0, len(df.columns), nrow)]
    summary_data.append(["All Columns", '\n'.join(all_columns)])  # Add list of all columns, nrow columns per row

    datetime_summary = [f"Total Datetime Features = {n_datetime}\n"] + [', '.join(datetime_columns[i:i+nrow]) for i in range(0, n_datetime, nrow)]
    summary_data.append(["Datetime Columns", '\n'.join(datetime_summary)]) 

    cat_summary = [f"'{col}': {df[col].nunique()} unique values" for col in categorical_columns]
    cat_summary.insert(0, f"Total Categorical Columns = {n_categorical}\n")
    summary_data.append(["Categorical Columns", '\n'.join(cat_summary)])  # Add list of categorical columns and their unique values

    num_summary = [f"Total Numerical Features = {n_numerical}\n"] + [', '.join(numerical_columns[i:i+nrow]) for i in range(0, len(numerical_columns), nrow)]
    summary_data.append(["Numerical Columns", '\n'.join(num_summary)])  # Add list of numerical columns, nrow features per row

    print("\nDescribe Data:\n")
    print(tabulate(summary_data, headers=["Feature Type", "Basic Brief"], tablefmt="grid")) # Print in a tabulated format

    # Print value counts for categorical columns with <= th_value_count values
    print(f"\nValue Counts of Categorical columns with less than or equal to {th_value_count} values:\n")
    for col in categorical_columns:
        if df[col].nunique() <= th_value_count:
            print(f"\tFeature {col}: {dict(sorted(df[col].value_counts().items(), key=lambda item: item[1], reverse=True))}") #Sorted dictionary

# Data_Brief(df) with default 10
Data_Brief(df, th_value_count = 10, nrow = 10) 


Describe Data:

+-----------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Feature Type                | Basic Brief                                                                                                                                                             |
+=============================+=========================================================================================================================================================================+
| Dimensions (rows x columns) | 786363 x 23                                                                                                                                                             |
+-----------------------------+------------------------------------------------------------------------------------------------------------------------------------------------

Insights:
1. The new dimension are 786363 x 23 (This will again change after categorical encoding).
2. There are 4 datetime features.
3. There are 9 Numerical features.
4. There are 10 Categorical features (if considered boolean under categorical type).
5. Out of the 10 Categorical features 'merchantName' has highest unique values.
6. Out of 10 Categorical features 8 features have less than 10 unique values, so we can able to do categorical encoding more logically for these features.
    (['acqCountry','merchantCountryCode','posEntryMode','posConditionCode','transactionType','cardPresent','expirationDateKeyInMatch','isFraud'])
7. Able to get idea on these 8 feature unique values and their count which we can use to calculate percentage if needed or further analysis or plots (I have also created some useful plots on this in file 2_Plot.ipynb)
	
	 acqCountry: {'US': 774709, 'MEX': 3130, 'CAN': 2424, 'PR': 1538}

	 merchantCountryCode: {'US': 778511, 'MEX': 3143, 'CAN': 2426, 'PR': 1559}
	
	 posEntryMode: {'05': 315035, '09': 236481, '02': 195934, '90': 19576, '80': 15283}

	 posConditionCode: {'01': 620725, '08': 147794, 1: 8062, '99': 7435, 8: 1840, 99: 98}

	 transactionType: {'PURCHASE': 745193, 'REVERSAL': 20303, 'ADDRESS_VERIFICATION': 20169}

	 cardPresent: {False: 433495, True: 352868}

	 expirationDateKeyInMatch: {False: 785320, True: 1043}
	
	 isFraud: {False: 773946, True: 12417}

## 5. Insights of Numerical Features using describe()

- The `describe()` function provides valuable statistics for the numerical features in the dataset:

In [18]:
df.describe().style.format('{:.3f}')  #use of style will render the summary statistics in a more easy-to-read format

,accountNumber,customerId,creditLimit,availableMoney,transactionAmount,cardCVV,enteredCVV,cardLast4Digits,currentBalance
count,786363.000,786363.000,786363.000,786363.000,786363.000,786363.000,786363.000,786363.000,786363.000
mean,537232599.462,537232599.462,10759.464,6250.725,136.986,544.467,544.184,4757.418,4508.739
std,255421092.307,255421092.307,11636.175,8880.784,147.726,261.524,261.551,2996.584,6457.442
min,100088067.000,100088067.000,250.000,-1005.630,0.000,100.000,0.000,0.000,0.000
25%,330133277.000,330133277.000,5000.000,1077.420,33.650,310.000,310.000,2178.000,689.910
50%,507456073.000,507456073.000,7500.000,3184.860,87.900,535.000,535.000,4733.000,2451.760
75%,767620004.000,767620004.000,15000.000,7500.000,191.480,785.000,785.000,7338.000,5291.095
max,999389635.000,999389635.000,50000.000,50000.000,2011.540,998.000,998.000,9998.000,47498.810


Insights:
1. From above the insights for accountNumber, and customerId isn't that much useful for obvious reason.
2. The credit limits for the customers vary widely, with the maximum being $50,000, the minimum being $250, and an average of $10,759.46. This indicates a diverse range of credit limits across the dataset.
3. Maximum availableMoney  is 50000, minimum is -1005.630, and average is 6250.725. The minimum of -1005.63, is intriguing and could indicate overutilization or issues with account balances.
4. The transactionAmount column has a wide range of values, with the maximum being $2011.54, the minimum being $0, and an average of $136.986. This suggests varying transaction sizes, including both smaller and larger amounts.
5. The currentBalance column also varies significantly, with the maximum being $47,498.81, the minimum being $0, and an average of $4508.739. This indicates that some customers maintain higher balances, while others keep their balances closer to zero. $0 suggests that some customers consistently pay off their balances.
6. The average currentBalance of $4508.74 implies that customers, on average, maintain a reasonable level of debt relative to their credit limits. 
7. The average availableMoney of $6250.73 suggests that customers typically have available funds above their average currentBalance ($4508.74).
8. Large discrepancies between creditLimit and currentBalance could signal potential credit risk if customers are utilizing a significant portion of their available credit.
9. The presence of 0 values in enteredCVV, cardLast4Digits attributes might suggest missing or incomplete data, as CVV values and cardLast4Digits are typically non-zero.
10. Mean of cardCVV and enteredCVV is approximately same indicate many of the customer entered correct CVV.  
11. Apart from these we also get insights about the 25th, 50th (median), and 75th percentiles along with the standard deviation provide insights into the distribution, central tendency, and dispersion of these numerical features.

# 6. Missing Values

In [19]:
def missing_data(df):
    # Get missing value counts and sort in descending order
    missing_counts = df.isnull().sum().sort_values(ascending=False)
    missing_counts = missing_counts[missing_counts > 0]
    missing_percentages = (missing_counts / df.shape[0]) * 100
    missing_info = pd.DataFrame({
        'Missing Count': missing_counts,
        'Missing Percentage': missing_percentages
    })
    print(missing_info) # Columns which have missing data along with counts and percentages
    return missing_info.index.to_list()

missing_columns = missing_data(df)

                     Missing Count  Missing Percentage
acqCountry                    4562            0.580139
posEntryMode                  4054            0.515538
merchantCountryCode            724            0.092069
transactionType                698            0.088763
posConditionCode               409            0.052012


In [20]:
missing_columns

['acqCountry',
 'posEntryMode',
 'merchantCountryCode',
 'transactionType',
 'posConditionCode']

** Note: I have worked on handling missing values later on under File 3_DataWrangling_Modeling.ipynb

In [21]:
if (df.accountNumber.nunique() - df.customerId.nunique())==0:
    print("Total number of unique accountNumber and customerId are same \ni.e. each customer have unique accountNumber")

Total number of unique accountNumber and customerId are same 
i.e. each customer have unique accountNumber


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786363 entries, 0 to 786362
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   accountNumber             786363 non-null  int64         
 1   customerId                786363 non-null  int64         
 2   creditLimit               786363 non-null  int64         
 3   availableMoney            786363 non-null  float64       
 4   transactionDateTime       786363 non-null  datetime64[ns]
 5   transactionAmount         786363 non-null  float64       
 6   merchantName              786363 non-null  object        
 7   acqCountry                781801 non-null  object        
 8   merchantCountryCode       785639 non-null  object        
 9   posEntryMode              782309 non-null  object        
 10  posConditionCode          785954 non-null  object        
 11  merchantCategoryCode      786363 non-null  object        
 12  cu

In [1]:
# to preserve the datetime data type instead of csv storing in hdf
df.to_hdf('data/df_transactions_continue.h5', key='data', mode='w')

Note: Continuing Further Analysis from file 2_Plot_Visualization.ipynb